In [1]:
import csv
import re
import numpy as np

save_data = []

with open('records_forever.csv', newline='') as csvfile:
    reader = csv.reader(csvfile)
    for row in reader:
        if len(row[1]) == 34 or len(row[1]) == 42:
            row[3] = row[3].lower()
            row[4] = row[4].lower()
            row[5] = str(row[5]).lower()
            row[5] = row[5].replace(",","")
            _description = row[5].split(" ")
            price = 0
            for word in _description:
                if '$' in word:
                    numbers = re.findall(r'\d+', word)
                    numbers = np.array(list(map(int, numbers)))
                    if len(numbers) > 0:
                        price = max(numbers)
            row.append(price)
            save_data.append(row)
save_data = save_data[1:]
print(save_data[0])

['2', '115p7UMMngoj1pMvkpHijcRdfJNXj6LrLn', '1', '', 'wannacry', 'https://bitcointalk.org/index.php?topic=1916199.0', '', 0]


In [12]:

import pandas as pd

save_df = pd.DataFrame(save_data)
save_df.columns = ['id','address', 'abuse_type_id', 'abuse_type_other', 'abuser', 'description', 'from_country','price']
# save_df = save_df.drop(['nan'], axis =1)
save_df = save_df.drop_duplicates('address')
save_df = save_df.fillna('0')

save_df.head(100)
print(save_df[save_df['address'] == '1J7dPHGmfyHpmNHGPg1dMo9KMrPYdBzSAV'])

          id                             address abuse_type_id  \
30374  34487  1J7dPHGmfyHpmNHGPg1dMo9KMrPYdBzSAV             4   

      abuse_type_other                                  abuser  \
30374                   joel bosler <lxnataliedkv@outlook.com>   

                                             description from_country  price  
30374  l?똢t me tell yo?똵 i a?똠tua?똪ly pla?똠?똢d a??ma?...      Hungary  10000  


In [3]:
keyword = "steal|pa‌ss|setup|install|mаliсiоus|tro‌ja‌n|keylogger|trojan|spoofed|password|pass|pass word|phishing|fishing|hack|Hack|spoofing|bitcointalk.org|jōke|jőke|friends|friend|pοrη|archive|privаte|­­­­p­­o­­r­­n­­|stu‌pi‌d|sеcret|recoŗding|pre‌y|warning|mistake|bomb|ca‌me‌ra‌|vi‌d‌eo‌|こんにちは|joke|a‌cqu‌a‌i‌nta‌nce‌s|relativęs|vids|personal|vi‌si‌te‌d|pedo|erotίc|se‌cre‌ts|ƿorn|dirt|videоtарe|porn|po‌rnogra‌phy|extort|videos|myself addr|stimulating|webcam|indecent|sorcha|lazaro|aaron|sex|porn|adult|video|vi‌deo‌|sextortian|tortian|tortion|Black|FBI|extorsion|fbi|extortion|threatening|threatens|threat|threatning"
check_description = save_df['description'].str.contains(keyword)
check_abuser = save_df['abuser'].str.contains(keyword)
check_abuse_type = save_df['abuse_type_other'].str.contains(keyword)
check_sum = check_description + check_abuser + check_abuse_type

sextortian_df = save_df[check_sum]
check_df = check_sum * -1
none_sextortian_condition = check_df == 0
none_sextortian_df = save_df[none_sextortian_condition]
print(len(save_df), len(sextortian_df), len(none_sextortian_df))

28964 22247 6717


In [4]:
keyword = "binance|funds|tumbler|fraud|mixing|mining|scam|invest|musk|vitalic|doubler|spam|SCAM|black|False"
check_description = none_sextortian_df['description'].str.contains(keyword)
check_abuser = none_sextortian_df['abuser'].str.contains(keyword)
check_abuse_type = none_sextortian_df['abuse_type_other'].str.contains(keyword)
check_sum = check_description + check_abuser + check_abuse_type

scam_df = none_sextortian_df[check_sum]
check_df = check_sum * -1
none_scam_condition = check_df == 0
none_scam_df = none_sextortian_df[none_scam_condition]
print(len(none_sextortian_df), len(scam_df), len(none_scam_df))

6717 2702 4015


In [5]:
keyword = "crypted|ransom|randsom|ransonware|malvware|malware|virus"
check_description = none_scam_df['description'].str.contains(keyword)
check_abuser = none_scam_df['abuser'].str.contains(keyword)
check_abuse_type = none_scam_df['abuse_type_other'].str.contains(keyword)
check_sum = check_description + check_abuser + check_abuse_type

hack_df = none_scam_df[check_sum]
check_df = check_sum * -1
none_hack_condition = check_df == 0
none_hack_df = none_scam_df[none_hack_condition]
print(len(none_scam_df), len(hack_df), len(none_hack_df))


4015 423 3592


In [6]:
check_description = none_hack_df['description'].str.contains("dark|Dark")
check_abuser = none_hack_df['abuser'].str.contains("dark|Dark")
check_abuse_type = none_hack_df['abuse_type_other'].str.contains("dark|Dark")
check_sum = check_description + check_abuser + check_abuse_type

dark_df = none_hack_df[check_sum]
check_df = check_sum * -1
none_dark_condition = check_df == 0
none_dark_df = none_hack_df[none_dark_condition]
print(len(none_hack_df), len(dark_df), len(none_dark_df))


3592 65 3527


In [7]:
sextortian_df['category'] = 'sextortian'
scam_df['category'] = 'scam'
hack_df['category'] = 'hack'
dark_df['category'] = 'darknet'
none_dark_df['category'] = 'other'

# save_df.columns = ['id','address', 'abuse_type_id', 'abuse_type_other', 'abuser', 'description', 'from_country','price']
merge_df = pd.concat([sextortian_df, scam_df, hack_df, dark_df, none_dark_df])
save_merge_df = pd.concat([merge_df['address'], merge_df['category'],merge_df['price']], axis = 1)
check_price = save_merge_df['price'] > 0
save_merge_df = save_merge_df[check_price]


# save_merge_df.to_csv('abuse_address.csv', header=False, index=False)

/tmp/ipykernel_113111/2519413260.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sextortian_df['category'] = 'sextortian'
/tmp/ipykernel_113111/2519413260.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  scam_df['category'] = 'scam'
/tmp/ipykernel_113111/2519413260.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/u